# CP423: Assignment 3

Group Number: 5

Group Members: Abigail Lee (200469770), Kayleigh Habib (200370580) and Myisha Chaudhry (200591740)

In [1]:
# MIGHT NEED TO RUN THIS FOR THE NLTK
#pip install certifi
# MIGHT NEED TO RUN THIS FOR reading file
# pip install lxml

In [2]:
import certifi
import ssl
import os

os.environ["SSL_CERT_FILE"] = certifi.where()

In [3]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Question 1 : Introduction to Webscraping

Your task involves working with the given URL, which pertains to historical population counts of various Canadian provinces as recorded on Wikipedia
(https://en.wikipedia.org/wiki/List_of_Canadian_provinces_and_territories_by_historical_population).

Your goal in this segment is to import HTML tables from the web and manipulate them using pandas.


Here's a breakdown of what you will do:

1. Retrieve the specified webpage as raw HTML using the requests library

2. Decode the HTML into a tree-structured Python object with the BeautifulSoup library

3. Utilize BeautifulSoup to identify and extract only the tables we're interested in

4. Merge the tables, sanitize the text, and transform them into a single Python dictionary

5. Construct a pandas dataframe out of this dictionary

6. Locate all h2 elements on the HTML page and display their text content

7. Generate a list of all the hyperlinks embedded within the tables

8. Download every webpage by traversing the links included in the list created in the previous step.

In [13]:
# Part 1: Retrieve the specified webpage as raw HTML
url = "https://en.wikipedia.org/wiki/List_of_Canadian_provinces_and_territories_by_historical_population"
response = requests.get(url) # retrieve the link
raw_html = response.text # get the contents as a string
# raw_html 

In [14]:
# Part 2: Decode the HTML into a tree-structure object
tree_stru = BeautifulSoup(raw_html, features = "html.parser") # html.parser converts it to a BeautifulSoup type
# tree_stru


In [15]:
# Part 3: Identify and extract only tables we are interested in
inter_tables = tree_stru.find_all('table',class_='wikitable') # find all tables
# inter_tables

In [7]:
def sanitize_func(data):
    result = ""
    pattern = "\[.+\]" # pattern
    result = re.sub(pattern," ", str(data)).strip() #if the pattern matches replace with empty string and strip any empty string

    return result

In [8]:
# Part 4: Merge, Sanitize and transform into Python dictionary
new_dict = {}
final_dict = {}
df_mer = pd.DataFrame() # empty merged df
index = 1
for table in inter_tables: # go through each table
    df = pd.read_html(str(table))[0] # read the html as a df of strings
    df = df.applymap(sanitize_func) # call the helper function
    
    # if it is the first occurence put all the data into our dataframe
    if index == 1:
        df_mer = df
    else: # otherwise merge them on the common field
        df_mer = df_mer.merge(df, on= 'Name', how = 'outer') #joins the tables on the names
    index+=1

#create a dictionary
new_dict = df_mer.to_dict()
# print(new_dict)

# go through each element and ensure it is cleaned
for key,val in new_dict.items():
    # key is the column name
    temp_dict = {}
    for k,v in val.items():
        # key is the index, value is the row
        temp_dict[k] = sanitize_func(str(v))

    final_dict[sanitize_func(key)] = temp_dict

print(final_dict)


{'Name': {0: 'Lower Canada', 1: 'New Brunswick', 2: 'Newfoundland', 3: 'Nova Scotia', 4: 'Prince Edward Island', 5: 'Upper Canada', 6: 'Total', 7: 'Alberta', 8: 'British Columbia', 9: 'Manitoba', 10: 'Newfoundland and Labrador', 11: 'Northwest Territories', 12: 'Nunavut', 13: 'Ontario', 14: 'Quebec', 15: 'Saskatchewan', 16: 'Yukon', 17: 'Canada'}, '1700': {0: '14000.0', 1: 'nan', 2: '500.0', 3: '1300.0', 4: 'nan', 5: 'nan', 6: '15800.0', 7: 'nan', 8: 'nan', 9: 'nan', 10: 'nan', 11: 'nan', 12: 'nan', 13: 'nan', 14: 'nan', 15: 'nan', 16: 'nan', 17: 'nan'}, '1725': {0: '29000.0', 1: 'nan', 2: '5000.0', 3: '5000.0', 4: '300.0', 5: 'nan', 6: '39300.0', 7: 'nan', 8: 'nan', 9: 'nan', 10: 'nan', 11: 'nan', 12: 'nan', 13: 'nan', 14: 'nan', 15: 'nan', 16: 'nan', 17: 'nan'}, '1750': {0: '54500.0', 1: 'nan', 2: '10000.0', 3: '14000.0', 4: '2500.0', 5: 'nan', 6: '81000.0', 7: 'nan', 8: 'nan', 9: 'nan', 10: 'nan', 11: 'nan', 12: 'nan', 13: 'nan', 14: 'nan', 15: 'nan', 16: 'nan', 17: 'nan'}, '1775': 

In [9]:
# Part 5: Construct a pandas dataframe
pd_df = pd.DataFrame(data = final_dict) # uses the dictionary as our data
pd_df.head() # prints first 5 rows


,Name,1700,1725,1750,1775,1800,1825,Confederated,1841,1851,...,1976,1981,1986,1991,1996,2001,2006,2011,2016,2021
0,Lower Canada,14000.0,29000.0,54500.0,96000.0,225000,450000,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,New Brunswick,nan,nan,nan,nan,10000,75000,1867.0,"156,162",193800,...,677250.0,689375.0,709442.0,723900.0,738133,729498,729997,751171,747101,775610
2,Newfoundland,500.0,5000.0,10000.0,16000.0,10000,45759,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,Nova Scotia,1300.0,5000.0,14000.0,20000.0,57000,150000,1867.0,"202,575",276854,...,828571.0,839805.0,873176.0,899942.0,909282,908007,913462,921727,923598,969383
4,Prince Edward Island,nan,300.0,2500.0,10000.0,20000,28600,1873.0,47042,"62,678",...,118229.0,121225.0,126646.0,129765.0,134557,135294,135851,140204,142907,154331


In [10]:
# Part 6: Locate h2 elements on the HTML page and display their text content

# find the h2 elements on the HTML page
h2_elements = tree_stru.find_all("h2")

for h2 in h2_elements:

    # get the h2 text content
    text = (h2.get_text())

    # remove the [edit] from the text
    text = text.replace('[edit]', '')
    print(text)


Contents
1700 to 1825
1841 to 1931
1941 to 1991
1996 to 2021
Notes


In [11]:
# Part 7: Generate a list of all hyperlinks embedded within the table

hyperlinks = []
names = []

for hl in inter_tables:

    # find all <a> elements in the table (contains the hyperlinks)
    links = table.find_all("a")

    for link in links:

        # get the href attribute of the <a> element which contains the URL of the hyperlink
        href = link.get("href")
        title = link.get("title")

        # check if it is a valid link and is a Wikipedia link
        if href and href.startswith('/wiki/'):

            # write out the full URL by adding the href element to https://en.wikipedia.org
            full_url = f"https://en.wikipedia.org{href}"

            # add the full URL to the hyperlinks list
            hyperlinks.append(full_url)
            names.append(title)

print(hyperlinks)


['https://en.wikipedia.org/wiki/Alberta', 'https://en.wikipedia.org/wiki/British_Columbia', 'https://en.wikipedia.org/wiki/Manitoba', 'https://en.wikipedia.org/wiki/New_Brunswick', 'https://en.wikipedia.org/wiki/Newfoundland_and_Labrador', 'https://en.wikipedia.org/wiki/Northwest_Territories', 'https://en.wikipedia.org/wiki/Nova_Scotia', 'https://en.wikipedia.org/wiki/Nunavut', 'https://en.wikipedia.org/wiki/Ontario', 'https://en.wikipedia.org/wiki/Prince_Edward_Island', 'https://en.wikipedia.org/wiki/Quebec', 'https://en.wikipedia.org/wiki/Saskatchewan', 'https://en.wikipedia.org/wiki/Yukon', 'https://en.wikipedia.org/wiki/Canada', 'https://en.wikipedia.org/wiki/Alberta', 'https://en.wikipedia.org/wiki/British_Columbia', 'https://en.wikipedia.org/wiki/Manitoba', 'https://en.wikipedia.org/wiki/New_Brunswick', 'https://en.wikipedia.org/wiki/Newfoundland_and_Labrador', 'https://en.wikipedia.org/wiki/Northwest_Territories', 'https://en.wikipedia.org/wiki/Nova_Scotia', 'https://en.wikipedi

In [12]:
# Part 8: Download every webpage by traversing links including in the list created in the previous step.
foldername = "webpages"

# create folder called "webpages"
os.makedirs(foldername, exist_ok=True)

index = 0

# loop through hyperlinks from question 7, add content of each page to folder
for link in hyperlinks:

  # get page content
  page = requests.get(link)
  webpage = page.content

  # set file name
  if names[index] is not None:
    filename = names[index]+'.html'

    # create new file with the hyperlink and add to folder
    path = os.path.join(foldername, filename)
    with open(path, 'wb') as f:
        f.write(webpage)

  index+=1


